# Climate Change Trend Analysis

Analyze long-term trends, variability, and projected changes in regional climate.

## Analysis Components

1. **Linear trends**: Temperature and precipitation changes over time
2. **Seasonal trends**: How seasons are changing differently
3. **Spatial patterns**: Where changes are largest
4. **Statistical significance**: Which trends are robust
5. **Multi-model agreement**: Ensemble consensus

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Setup complete")

## Load Bias-Corrected Data

In [ ]:
# Load from previous notebook (or create synthetic)
# For demonstration, we'll create a synthetic future time series with trend

years = np.arange(2020, 2100)
n_years = len(years)

# Create monthly data with warming trend
months_per_year = 12
time = pd.date_range('2020-01-01', periods=n_years*months_per_year, freq='MS')

# Baseline temperature with seasonal cycle
baseline = 15
seasonal = 10 * np.sin(2 * np.pi * np.arange(len(time)) / 12)

# Add warming trend: 0.03°C/year (3°C by 2100)
trend_rate = 0.03  # °C per year
years_array = np.repeat(np.arange(n_years), months_per_year)
trend = trend_rate * years_array

# Add natural variability
variability = np.random.normal(0, 0.5, len(time))

temperature = baseline + seasonal + trend + variability

# Create dataset
ds = xr.Dataset({
    'tas': (['time'], temperature)
}, coords={'time': time})

ds['tas'].attrs = {'units': 'degC', 'long_name': 'Near-surface air temperature'}

print(f"Data period: {ds.time.dt.year.min().values} - {ds.time.dt.year.max().values}")
print(f"Mean temperature: {ds['tas'].mean().values:.2f}°C")

## Calculate Linear Trends

Use least-squares regression to estimate temperature change rates.

In [ ]:
# Calculate annual means
ds_annual = ds.groupby('time.year').mean()

# Linear regression
years_num = ds_annual.year.values
tas_annual = ds_annual['tas'].values

slope, intercept, r_value, p_value, std_err = stats.linregress(years_num, tas_annual)

# Convert to per-decade rate
trend_per_decade = slope * 10
total_change = slope * (years_num[-1] - years_num[0])

print("Linear Trend Analysis:")
print(f"Trend: {trend_per_decade:.3f}°C/decade")
print(f"Total change ({years_num[0]}-{years_num[-1]}): {total_change:.2f}°C")
print(f"R²: {r_value**2:.3f}")
print(f"p-value: {p_value:.2e}")
print(f"Significance: {'Yes' if p_value < 0.05 else 'No'} (p < 0.05)")

## Visualize Long-Term Trends

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Annual time series with trend line
axes[0, 0].plot(years_num, tas_annual, 'o-', alpha=0.6, label='Annual mean')
axes[0, 0].plot(years_num, intercept + slope * years_num, 'r-', linewidth=2, 
                label=f'Trend: {trend_per_decade:.2f}°C/decade')
axes[0, 0].fill_between(years_num, 
                       intercept + slope * years_num - 2*std_err*years_num,
                       intercept + slope * years_num + 2*std_err*years_num,
                       alpha=0.2, color='red')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].set_title('Annual Mean Temperature Trend')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Monthly time series with 5-year running mean
tas_5yr = ds['tas'].rolling(time=60, center=True).mean()
axes[0, 1].plot(ds.time, ds['tas'], alpha=0.2, linewidth=0.5, color='gray', label='Monthly')
axes[0, 1].plot(tas_5yr.time, tas_5yr, 'b-', linewidth=2, label='5-year mean')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Temperature (°C)')
axes[0, 1].set_title('Monthly Temperature with 5-Year Running Mean')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Seasonal trends
seasons = {'DJF': [12, 1, 2], 'MAM': [3, 4, 5], 'JJA': [6, 7, 8], 'SON': [9, 10, 11]}
season_trends = {}

for season_name, months in seasons.items():
    season_data = ds.sel(time=ds.time.dt.month.isin(months))
    season_annual = season_data.groupby('time.year').mean()
    slope_s, _, _, _, _ = stats.linregress(season_annual.year.values, season_annual['tas'].values)
    season_trends[season_name] = slope_s * 10

axes[1, 0].bar(season_trends.keys(), season_trends.values(), color=['blue', 'green', 'red', 'orange'],
              alpha=0.7, edgecolor='black')
axes[1, 0].set_ylabel('Trend (°C/decade)')
axes[1, 0].set_title('Seasonal Temperature Trends')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Decadal changes
ds['decade'] = (ds.time.dt.year // 10) * 10
decadal_means = ds.groupby('decade').mean()

axes[1, 1].plot(decadal_means.decade, decadal_means['tas'], 'o-', linewidth=2, markersize=10)
for i, (dec, temp) in enumerate(zip(decadal_means.decade.values, decadal_means['tas'].values)):
    axes[1, 1].text(dec, temp, f"{temp:.1f}°C", ha='center', va='bottom', fontsize=9)
axes[1, 1].set_xlabel('Decade')
axes[1, 1].set_ylabel('Temperature (°C)')
axes[1, 1].set_title('Decadal Mean Temperature')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Change by Time Period

In [ ]:
# Compare different 30-year periods
periods = {
    '2020-2049': slice('2020', '2049'),
    '2040-2069': slice('2040', '2069'),
    '2070-2099': slice('2070', '2099')
}

period_stats = []
for name, period in periods.items():
    data = ds.sel(time=period)
    period_stats.append({
        'Period': name,
        'Mean': data['tas'].mean().values,
        'Std': data['tas'].std().values,
        'Min': data['tas'].min().values,
        'Max': data['tas'].max().values
    })

stats_df = pd.DataFrame(period_stats)
print("\nTemperature Statistics by Period:")
print(stats_df.round(2))

# Calculate changes
ref_mean = stats_df.iloc[0]['Mean']
print(f"\nChange from 2020-2049:")
for i in range(1, len(stats_df)):
    change = stats_df.iloc[i]['Mean'] - ref_mean
    print(f"  {stats_df.iloc[i]['Period']}: +{change:.2f}°C")

## Summary

### Key Findings

1. **Warming Rate**: Clear upward trend throughout 21st century
2. **Seasonal Variation**: All seasons showing warming, with potential differences in magnitude
3. **Acceleration**: Later periods show higher temperatures than early century
4. **Statistical Significance**: Trends are robust (p < 0.05)

### Next Steps

- **Notebook 04**: Analyze extreme events (heat waves, cold spells)
- **Notebook 05**: Assess sector-specific impacts
- **Notebook 06**: Create final visualizations and reports